In [45]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.cross_validation import ShuffleSplit, KFold

import xgboost as xgb

In [2]:
transactions = pd.read_csv('transactions.csv')
customers_gender = pd.read_csv('customers_gender_train.csv')

In [4]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [5]:
customers_gender.head()

,customer_id,gender
0,75562265,0
1,10928546,1
2,69348468,1
3,84816985,1
4,61009479,0


In [14]:
X = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

In [17]:
X.head()

,742,1711,1731,1799,2741,3000,3351,3501,4111,4112,...,8299,8398,8641,8699,8999,9211,9222,9311,9399,9402
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
31385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
customers_gender = customers_gender.set_index('customer_id')

In [16]:
customers_gender.head()

,gender
customer_id,
75562265,0
10928546,1
69348468,1
84816985,1
61009479,0


In [31]:
Y_tr = customers_gender.loc[X.index].gender
Y_tr = Y_tr.reset_index()
del Y_tr['customer_id']
Y_tr = Y_tr.dropna(0)

In [32]:
X_tr = X.reset_index()
X_tr = X_tr.loc[Y_train.index].set_index('customer_id')

In [63]:
for itr, ite in ShuffleSplit(X_tr.shape[0], n_iter=4, train_size=0.7, test_size=0.3, random_state=0):
    pass

In [64]:
X_train = X_tr.iloc[itr]
X_val = X_tr.iloc[ite]

Y_train = Y_tr.iloc[itr]
Y_val = Y_tr.iloc[ite]

### Sklearn GradientBoostingClassifier

In [65]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train.values[:, 0])

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=13, subsample=1.0, verbose=0,
              warm_start=False)

In [66]:
predicted = clf.predict_proba(X_val)[:, 1]

In [67]:
auc_score = roc_auc_score(Y_val, predicted)
print "Auc score %s" % str(auc_score) 

Auc score 0.855550482504


## XGboost

In [56]:
%%time
model = xgb.XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=250, silent=True, \
                          objective='binary:logistic', nthread=24, \
                          subsample=0.6, colsample_bytree=0.6, colsample_bylevel=0.7)
model.fit(X_train, Y_train.gender)

CPU times: user 21.9 s, sys: 207 ms, total: 22.1 s
Wall time: 22.3 s


In [57]:
predicted = model.predict_proba(X_val)[:, 1]

In [58]:
auc_score = roc_auc_score(Y_val, predicted)
print "Auc score %s" % str(auc_score) 

Auc score 0.873794911787


In [138]:
param = {}
param['max_depth'] = 5
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'auc'
param['eta'] = 0.1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['colsample_bylevel'] = 0.7

numround = 250

In [139]:
Xdatatrain = xgb.DMatrix(X_train, label = Y_train)
Xdatatest = xgb.DMatrix(X_val, label = Y_val)

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdatatest, 'eval')]            

bst = xgb.train(plst, Xdatatrain, numround, evals = watchlist, verbose_eval = 10)
y_pred = bst.predict(Xdatatest)

[0]	train-auc:0.737894	eval-auc:0.728088
[10]	train-auc:0.848071	eval-auc:0.819398
[20]	train-auc:0.871421	eval-auc:0.833069
[30]	train-auc:0.88578	eval-auc:0.843194
[40]	train-auc:0.895822	eval-auc:0.848619
[50]	train-auc:0.90491	eval-auc:0.852659
[60]	train-auc:0.91158	eval-auc:0.856331
[70]	train-auc:0.917526	eval-auc:0.859297
[80]	train-auc:0.921446	eval-auc:0.861239
[90]	train-auc:0.925058	eval-auc:0.862544
[100]	train-auc:0.92873	eval-auc:0.863985
[110]	train-auc:0.931368	eval-auc:0.865349
[120]	train-auc:0.933405	eval-auc:0.866298
[130]	train-auc:0.935349	eval-auc:0.867066
[140]	train-auc:0.937994	eval-auc:0.867286
[150]	train-auc:0.940376	eval-auc:0.867714
[160]	train-auc:0.941617	eval-auc:0.867894
[170]	train-auc:0.94388	eval-auc:0.868635
[180]	train-auc:0.945917	eval-auc:0.868798
[190]	train-auc:0.947791	eval-auc:0.868741
[200]	train-auc:0.949481	eval-auc:0.868943
[210]	train-auc:0.95171	eval-auc:0.868574
[220]	train-auc:0.953848	eval-auc:0.868405
[230]	train-auc:0.95524	eval

In [140]:
print "Auc score %s" % str(roc_auc_score(Y_val, y_pred)) 

Auc score 0.868003572431


In [144]:
Xdatatrain = xgb.DMatrix(X_tr, label = Y_tr)
bst = xgb.train(plst, Xdatatrain, numround, verbose_eval = 10)

X_test = X.drop(customers_gender.index)
Xtest = xgb.DMatrix(X_test)

res = bst.predict(Xtest)

In [147]:
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = res

In [148]:
result.head()

,customer_id,gender
0,49101,0.369926
1,114348,0.696570
2,127946,0.856614
3,137367,0.427838
4,174467,0.726184


In [149]:
result.to_csv('baseline_xgb.csv', index=False)

In [51]:
X_test = X.drop(customers_gender.index)
print len(X_test)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

3000
